# 1. Carga y Unión de Datos (Data Wrangling)

## Objetivo
Construir un solo dataset limpio y coherente con los nacimientos de Guatemala del período 2009–2022.

## Fuente de datos
- **Origen:** Instituto Nacional de Estadística (INE) de Guatemala
- **Formato:** Archivos SPSS (.sav)
- **Período:** 2009 a 2022 (14 años)
- **Unidad de análisis:** Cada registro individual de nacimiento

---

## 1.1 Importación de librerías

In [2]:
import pyreadstat
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("Librerías cargadas correctamente")

Librerías cargadas correctamente


## 1.2 Definición de rutas y archivos

In [3]:
# Ruta a la carpeta de datos
DATA_PATH = Path('../data')

# Listar archivos .sav disponibles
archivos_sav = sorted([f for f in os.listdir(DATA_PATH) if f.endswith('.sav')])

print(f"Carpeta de datos: {DATA_PATH.resolve()}")
print(f"\nArchivos .sav encontrados: {len(archivos_sav)}")
print("-" * 40)
for archivo in archivos_sav:
    print(f"  • {archivo}")

Carpeta de datos: C:\Users\aeh55\Documents\Angel Esteban\Universidad\Semestre 7\MineríadeDatos\PRY1-MD\data

Archivos .sav encontrados: 14
----------------------------------------
  • N2009.sav
  • N2010.sav
  • N2011.sav
  • N2012.sav
  • N2013.sav
  • N2014.sav
  • N2015.sav
  • N2016.sav
  • N2017.sav
  • N2018.sav
  • N2019.sav
  • N2020.sav
  • N2021.sav
  • N2022.sav


## 1.3 Carga individual de archivos y verificación de estructura

Cargamos cada archivo y extraemos información sobre:
- Número de registros
- Variables (columnas)
- Año de los datos

In [4]:
# Diccionario para almacenar información de cada archivo
info_archivos = {}
dataframes = {}
metadatos = {}

print("Cargando archivos...\n")
print(f"{'Archivo':<15} {'Registros':>12} {'Variables':>12} {'Año datos':>12}")
print("=" * 55)

for archivo in archivos_sav:
    filepath = DATA_PATH / archivo
    
    # Cargar archivo
    df, meta = pyreadstat.read_sav(str(filepath))
    
    # Extraer año esperado del nombre
    año_nombre = int(archivo.replace('N', '').replace('.sav', ''))
    
    # Determinar año real de los datos
    if 'Añoocu' in df.columns:
        año_datos = int(df['Añoocu'].dropna().mode().iloc[0])
        if año_datos < 100:
            año_datos = 2000 + año_datos
    elif 'Añoreg' in df.columns:
        año_datos = int(df['Añoreg'].dropna().mode().iloc[0])
    else:
        año_datos = año_nombre
    
    # Guardar información
    info_archivos[año_nombre] = {
        'archivo': archivo,
        'registros': len(df),
        'variables': len(df.columns),
        'columnas': list(df.columns),
        'año_datos': año_datos
    }
    
    dataframes[año_nombre] = df
    metadatos[año_nombre] = meta
    
    # Mostrar información
    estado = "✓" if año_nombre == año_datos else "⚠"
    print(f"{archivo:<15} {len(df):>12,} {len(df.columns):>12} {año_datos:>12} {estado}")

print("=" * 55)
print(f"\nTotal de archivos cargados: {len(dataframes)}")

Cargando archivos...

Archivo            Registros    Variables    Año datos
N2009.sav            351,628           37         2009 ✓
N2010.sav            361,906           39         2010 ✓
N2011.sav            373,692           39         2011 ✓
N2012.sav            388,613           41         2012 ✓
N2013.sav            387,342           41         2013 ✓
N2014.sav            386,195           39         2014 ✓
N2015.sav            391,425           42         2015 ✓
N2016.sav            390,382           42         2016 ✓
N2017.sav            381,664           42         2017 ✓
N2018.sav            383,263           42         2018 ✓
N2019.sav            366,855           42         2019 ✓
N2020.sav            341,212           42         2020 ✓
N2021.sav            345,149           42         2021 ✓
N2022.sav            345,869           42         2022 ✓

Total de archivos cargados: 14


## 1.4 Análisis de variables por archivo

Verificamos si todos los archivos tienen las mismas variables con los mismos nombres.

In [6]:
# Obtener todas las variables únicas
todas_variables = set()
for año, info in info_archivos.items():
    todas_variables.update(info['columnas'])

print(f"Total de variables únicas encontradas: {len(todas_variables)}")
print("\n" + "=" * 80)
print("MATRIZ DE PRESENCIA DE VARIABLES POR AÑO")
print("=" * 80)

# Crear matriz de presencia
años = sorted(info_archivos.keys())
presencia = {}

for var in sorted(todas_variables):
    presencia[var] = []
    for año in años:
        if var in info_archivos[año]['columnas']:
            presencia[var].append('✓')
        else:
            presencia[var].append('-')

# Mostrar como DataFrame
df_presencia = pd.DataFrame(presencia, index=años).T
df_presencia.columns = [str(a) for a in años]
print(df_presencia.to_string())

Total de variables únicas encontradas: 61

MATRIZ DE PRESENCIA DE VARIABLES POR AÑO
         2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022
Areag       ✓    ✓    ✓    -    -    -    -    -    -    -    -    -    -    -
Asisrec     ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓
Añoocu      ✓    ✓    ✓    -    -    -    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓
Añoreg      ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓
Ciuomad     -    -    -    -    ✓    -    -    -    -    -    -    -    -    -
Ciuopad     -    -    -    -    ✓    -    -    -    -    -    -    -    -    -
Depnam      ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓
Depnap      ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓
Depocu      ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓
Depreg      ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓    ✓
Deprem      ✓    ✓    ✓    ✓    ✓    ✓    ✓    

In [7]:
# Identificar variables comunes a todos los años
variables_comunes = todas_variables.copy()
for año, info in info_archivos.items():
    variables_comunes = variables_comunes.intersection(set(info['columnas']))

print(f"Variables presentes en TODOS los años: {len(variables_comunes)}")
print("-" * 50)
for var in sorted(variables_comunes):
    print(f"  • {var}")

# Variables que no están en todos los años
variables_parciales = todas_variables - variables_comunes
print(f"\nVariables presentes solo en ALGUNOS años: {len(variables_parciales)}")
print("-" * 50)
for var in sorted(variables_parciales):
    años_presente = [str(a) for a in años if var in info_archivos[a]['columnas']]
    print(f"  • {var}: {', '.join(años_presente)}")

Variables presentes en TODOS los años: 24
--------------------------------------------------
  • Asisrec
  • Añoreg
  • Depnam
  • Depnap
  • Depocu
  • Depreg
  • Deprem
  • Deprep
  • Diaocu
  • Edadm
  • Edadp
  • Escivm
  • Escivp
  • Libras
  • Mesocu
  • Mesreg
  • Mupocu
  • Onzas
  • Sexo
  • Sitioocu
  • Tipar
  • Tohinm
  • Tohite
  • Tohivi

Variables presentes solo en ALGUNOS años: 37
--------------------------------------------------
  • Areag: 2009, 2010, 2011
  • Añoocu: 2009, 2010, 2011, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022
  • Ciuomad: 2013
  • Ciuopad: 2013
  • Escolam: 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022
  • Escolap: 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022
  • Gretnm: 2009
  • Gretnp: 2009, 2010, 2011, 2012
  • Munpnap: 2014, 2015, 2016, 2017, 2018, 2019, 2020
  • Mupnam: 2009, 2011, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022
  • Mupnap: 2009, 2011, 2012, 2013, 202

## 1.5 Análisis de tipos de datos

Verificamos que las variables comunes tengan tipos de datos consistentes.

In [8]:
# Analizar tipos de datos de las variables comunes
print("TIPOS DE DATOS POR VARIABLE Y AÑO")
print("=" * 80)

tipos_por_variable = {}
for var in sorted(variables_comunes):
    tipos = {}
    for año in años:
        if var in dataframes[año].columns:
            tipos[año] = str(dataframes[año][var].dtype)
    tipos_por_variable[var] = tipos

# Identificar inconsistencias
inconsistencias = []
for var, tipos in tipos_por_variable.items():
    tipos_unicos = set(tipos.values())
    if len(tipos_unicos) > 1:
        inconsistencias.append((var, tipos_unicos))

if inconsistencias:
    print(f"\n⚠️ Variables con tipos inconsistentes: {len(inconsistencias)}")
    for var, tipos in inconsistencias:
        print(f"  • {var}: {tipos}")
else:
    print("\n✓ Todas las variables comunes tienen tipos consistentes")

# Mostrar tipos de las variables comunes (tomando el primer año como referencia)
print("\nTipos de datos de variables comunes (referencia año 2009):")
print("-" * 50)
df_ref = dataframes[2009]
for var in sorted(variables_comunes):
    if var in df_ref.columns:
        print(f"  {var:<20} {str(df_ref[var].dtype):<15}")

TIPOS DE DATOS POR VARIABLE Y AÑO

⚠️ Variables con tipos inconsistentes: 1
  • Mupocu: {'str', 'float64'}

Tipos de datos de variables comunes (referencia año 2009):
--------------------------------------------------
  Asisrec              float64        
  Añoreg               float64        
  Depnam               float64        
  Depnap               float64        
  Depocu               float64        
  Depreg               float64        
  Deprem               float64        
  Deprep               float64        
  Diaocu               float64        
  Edadm                float64        
  Edadp                float64        
  Escivm               float64        
  Escivp               float64        
  Libras               float64        
  Mesocu               float64        
  Mesreg               float64        
  Mupocu               str            
  Onzas                float64        
  Sexo                 float64        
  Sitioocu             float64        
  

## 1.6 Unión de los datasets

Procedemos a unir todos los archivos en un único DataFrame, utilizando solo las variables comunes para garantizar coherencia.

In [9]:
# Preparar lista de DataFrames para concatenar
dfs_para_unir = []

for año in años:
    df_temp = dataframes[año].copy()
    
    # Agregar columna de año de origen si no existe Añoocu
    if 'Añoocu' not in df_temp.columns:
        df_temp['Añoocu'] = año
    
    # Seleccionar solo variables comunes + Añoocu
    cols_seleccionar = list(variables_comunes)
    if 'Añoocu' not in cols_seleccionar:
        cols_seleccionar.append('Añoocu')
    
    # Filtrar columnas existentes
    cols_existentes = [c for c in cols_seleccionar if c in df_temp.columns]
    df_temp = df_temp[cols_existentes]
    
    dfs_para_unir.append(df_temp)
    print(f"Año {año}: {len(df_temp):>10,} registros preparados")

# Concatenar todos los DataFrames
print("\nUniendo datasets...")
df_completo = pd.concat(dfs_para_unir, ignore_index=True)
print(f"\n✓ Dataset unificado creado exitosamente")

Año 2009:    351,628 registros preparados
Año 2010:    361,906 registros preparados
Año 2011:    373,692 registros preparados
Año 2012:    388,613 registros preparados
Año 2013:    387,342 registros preparados
Año 2014:    386,195 registros preparados
Año 2015:    391,425 registros preparados
Año 2016:    390,382 registros preparados
Año 2017:    381,664 registros preparados
Año 2018:    383,263 registros preparados
Año 2019:    366,855 registros preparados
Año 2020:    341,212 registros preparados
Año 2021:    345,149 registros preparados
Año 2022:    345,869 registros preparados

Uniendo datasets...

✓ Dataset unificado creado exitosamente


## 1.7 Verificación del dataset unificado

In [10]:
print("=" * 60)
print("RESUMEN DEL DATASET UNIFICADO")
print("=" * 60)

print(f"\n📊 Dimensiones:")
print(f"   • Total de registros (nacimientos): {len(df_completo):,}")
print(f"   • Total de variables: {len(df_completo.columns)}")

# Verificar rango de años
if 'Añoocu' in df_completo.columns:
    df_completo['Añoocu'] = pd.to_numeric(df_completo['Añoocu'], errors='coerce')
    # Normalizar años (si están en formato corto)
    df_completo.loc[df_completo['Añoocu'] < 100, 'Añoocu'] = df_completo.loc[df_completo['Añoocu'] < 100, 'Añoocu'] + 2000
    
    año_min = int(df_completo['Añoocu'].min())
    año_max = int(df_completo['Añoocu'].max())
    print(f"\n📅 Período de datos:")
    print(f"   • Año mínimo: {año_min}")
    print(f"   • Año máximo: {año_max}")
    print(f"   • Rango: {año_max - año_min + 1} años")

RESUMEN DEL DATASET UNIFICADO

📊 Dimensiones:
   • Total de registros (nacimientos): 5,195,195
   • Total de variables: 25

📅 Período de datos:
   • Año mínimo: 2009
   • Año máximo: 2022
   • Rango: 14 años


In [11]:
# Distribución de registros por año
print("\n📈 Distribución de nacimientos por año:")
print("-" * 45)

registros_por_año = df_completo['Añoocu'].value_counts().sort_index()

for año, registros in registros_por_año.items():
    barra = '█' * int(registros / 10000)
    print(f"   {int(año)}: {registros:>10,}  {barra}")

print("-" * 45)
print(f"   TOTAL: {registros_por_año.sum():>10,}")


📈 Distribución de nacimientos por año:
---------------------------------------------
   2009:    351,628  ███████████████████████████████████
   2010:    361,906  ████████████████████████████████████
   2011:    373,692  █████████████████████████████████████
   2012:    388,613  ██████████████████████████████████████
   2013:    387,342  ██████████████████████████████████████
   2014:    386,195  ██████████████████████████████████████
   2015:    391,425  ███████████████████████████████████████
   2016:    390,382  ███████████████████████████████████████
   2017:    381,664  ██████████████████████████████████████
   2018:    383,263  ██████████████████████████████████████
   2019:    366,855  ████████████████████████████████████
   2020:    341,212  ██████████████████████████████████
   2021:    345,149  ██████████████████████████████████
   2022:    345,869  ██████████████████████████████████
---------------------------------------------
   TOTAL:  5,195,195


In [12]:
# Verificar consistencia: suma de archivos individuales vs dataset unificado
suma_individual = sum(info['registros'] for info in info_archivos.values())
total_unificado = len(df_completo)

print("\n🔍 Verificación de integridad:")
print(f"   • Suma de registros individuales: {suma_individual:,}")
print(f"   • Total en dataset unificado: {total_unificado:,}")

if suma_individual == total_unificado:
    print("   ✓ Los totales coinciden - Unión correcta")
else:
    diferencia = abs(suma_individual - total_unificado)
    print(f"   ⚠️ Diferencia de {diferencia:,} registros")


🔍 Verificación de integridad:
   • Suma de registros individuales: 5,195,195
   • Total en dataset unificado: 5,195,195
   ✓ Los totales coinciden - Unión correcta


In [13]:
# Vista previa del dataset
print("\n📋 Vista previa del dataset (primeras 5 filas):")
print("=" * 80)
df_completo.head()


📋 Vista previa del dataset (primeras 5 filas):


,Diaocu,Depnap,Sexo,Depreg,Deprep,Tohivi,Depocu,Edadp,Deprem,Mesreg,Mupocu,Tohinm,Tipar,Mesocu,Escivp,Libras,Tohite,Onzas,Añoreg,Asisrec,Depnam,Sitioocu,Escivm,Edadm,Añoocu
0,28.0,1.0,1.0,1.0,1.0,2.0,1.0,30.0,1.0,8.0,0101,0.0,1.0,6.0,1.0,5.0,2.0,2.0,9.0,1.0,1.0,4.0,2.0,28.0,2009.0
1,2.0,1.0,1.0,1.0,1.0,1.0,1.0,23.0,1.0,3.0,0101,0.0,1.0,4.0,2.0,5.0,1.0,6.0,10.0,1.0,1.0,4.0,2.0,18.0,2009.0
2,30.0,1.0,2.0,1.0,1.0,3.0,1.0,32.0,1.0,2.0,0101,0.0,1.0,1.0,2.0,5.0,3.0,10.0,9.0,1.0,1.0,4.0,2.0,40.0,2009.0
3,7.0,1.0,2.0,1.0,1.0,2.0,1.0,30.0,1.0,1.0,0101,0.0,1.0,9.0,1.0,6.0,2.0,0.0,10.0,1.0,1.0,4.0,1.0,28.0,2009.0
4,10.0,1.0,1.0,1.0,1.0,2.0,1.0,33.0,1.0,7.0,0101,0.0,1.0,7.0,1.0,6.0,2.0,10.0,9.0,1.0,1.0,4.0,1.0,34.0,2009.0


In [14]:
# Información general del dataset
print("\n📊 Información del dataset:")
print("=" * 80)
df_completo.info()


📊 Información del dataset:
<class 'pandas.DataFrame'>
RangeIndex: 5195195 entries, 0 to 5195194
Data columns (total 25 columns):
 #   Column    Dtype  
---  ------    -----  
 0   Diaocu    float64
 1   Depnap    float64
 2   Sexo      float64
 3   Depreg    float64
 4   Deprep    float64
 5   Tohivi    float64
 6   Depocu    float64
 7   Edadp     float64
 8   Deprem    float64
 9   Mesreg    float64
 10  Mupocu    object 
 11  Tohinm    float64
 12  Tipar     float64
 13  Mesocu    float64
 14  Escivp    float64
 15  Libras    float64
 16  Tohite    float64
 17  Onzas     float64
 18  Añoreg    float64
 19  Asisrec   float64
 20  Depnam    float64
 21  Sitioocu  float64
 22  Escivm    float64
 23  Edadm     float64
 24  Añoocu    float64
dtypes: float64(24), object(1)
memory usage: 990.9+ MB


## 1.8 Guardar dataset unificado

Guardamos el dataset procesado para su uso en análisis posteriores.

In [15]:
# Crear carpeta de datos procesados si no existe
PROCESSED_PATH = Path('../data/processed')
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

# Guardar en formato CSV 
output_csv = PROCESSED_PATH / 'nacimientos_2009_2022.csv'
df_completo.to_csv(output_csv, index=False)
print(f"✓ Dataset guardado en: {output_csv}")
print(f"  Tamaño del archivo: {output_csv.stat().st_size / (1024**2):.2f} MB")
print(f"  Total de registros: {len(df_completo):,}")
print(f"  Total de variables: {len(df_completo.columns)}")

✓ Dataset guardado en: ..\data\processed\nacimientos_2009_2022.csv
  Tamaño del archivo: 569.82 MB
  Total de registros: 5,195,195
  Total de variables: 25


## 1.9 Resumen de la etapa de carga y unión

### Documentación del proceso

In [16]:
print("")
print("╔" + "═" * 58 + "╗")
print("║" + " RESUMEN: CARGA Y UNIÓN DE DATOS ".center(58) + "║")
print("╠" + "═" * 58 + "╣")
print("║" + " " * 58 + "║")
print(f"║  📁 Archivos procesados: {len(archivos_sav):<30} ║")
print(f"║  📅 Período: 2009 - 2022 ({año_max - año_min + 1} años){' ' * 21}║")
print(f"║  📊 Total de registros: {len(df_completo):,}{' ' * (32 - len(f'{len(df_completo):,}'))}║")
print(f"║  📋 Variables comunes: {len(variables_comunes):<32} ║")
print("║" + " " * 58 + "║")
print("║  ✓ Datos verificados y unificados correctamente" + " " * 9 + "║")
print("║" + " " * 58 + "║")
print("╚" + "═" * 58 + "╝")


╔══════════════════════════════════════════════════════════╗
║             RESUMEN: CARGA Y UNIÓN DE DATOS              ║
╠══════════════════════════════════════════════════════════╣
║                                                          ║
║  📁 Archivos procesados: 14                             ║
║  📅 Período: 2009 - 2022 (14 años)                     ║
║  📊 Total de registros: 5,195,195                       ║
║  📋 Variables comunes: 24                               ║
║                                                          ║
║  ✓ Datos verificados y unificados correctamente         ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝


In [ ]:
# Crear diccionario de metadatos para documentación
metadatos_proceso = {
    'fecha_procesamiento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'archivos_fuente': archivos_sav,
    'total_archivos': len(archivos_sav),
    'periodo': {'inicio': año_min, 'fin': año_max},
    'total_registros': len(df_completo),
    'variables_comunes': sorted(list(variables_comunes)),
    'variables_parciales': sorted(list(variables_parciales)),
    'registros_por_año': registros_por_año.to_dict(),
    'archivos_generados': [
        str(output_csv)
    ],
}

# Guardar metadatos
import json
with open(PROCESSED_PATH / 'metadatos_carga.json', 'w', encoding='utf-8') as f:
    json.dump(metadatos_proceso, f, indent=2, ensure_ascii=False, default=str)

print("✓ Metadatos del proceso guardados en: metadatos_carga.json")

NameError: name 'output_file' is not defined

---

## Notas para la sección "Descripción de los datos"

Este notebook documenta:

1. **Fuente de datos:** 14 archivos SPSS (.sav) del INE de Guatemala
2. **Período cubierto:** 2009 a 2022
3. **Unidad de análisis:** Cada registro representa un nacimiento individual
4. **Variables disponibles:** Se identificaron las variables comunes a todos los años
5. **Proceso de unión:** Concatenación vertical de todos los archivos
6. **Verificación:** Se validó la integridad de los datos comparando totales

El dataset resultante está listo para las siguientes etapas de análisis exploratorio.